### Notebook to test model for final performance metrics

In [12]:
import sys
import os
from matplotlib import pyplot as plt
from tqdm import tqdm
import torch
sys.path.append(os.path.dirname(os.path.abspath('.')))

from dev.utils import load_wandb_model

model, cfg, train_loader, val_loader, test_loader, topo_data, metadata = load_wandb_model('n7vcocod', name='epoch=5.ckpt', device='cuda')


Dataloaders and Topology Data Overview:
Batch size: 4
----------------------------------------------------------
Name                 | Samples      | Batches     
----------------------------------------------------------
Train Loader         | 6111         | 1527        
Val Loader           | 873          | 218         
Test Loader          | 1746         | 436         

Topology Data:
Number of nodes: 437
Number of edges: 5460
----------------------------------------------------------
Imported model from run "n7vcocod".
  | Name                | Type              | Params
----------------------------------------------------------
0 | criterion           | L1Loss            | 0     
1 | input_encoder_patch | Linear            | 384   
2 | input_encoder_node  | Linear            | 384   
3 | edge_encoder        | Linear            | 256   
4 | gnns                | ModuleList        | 99.6 K
5 | U                   | ModuleList        | 16.8 K
6 | mixer_patch         | MLPMixerTempo

In [13]:
# Run test with Lightning trainer
from lightning import Trainer

trainer = Trainer(
    accelerator='auto',
    devices=1,
    logger=None,
    enable_checkpointing=False
)
torch.set_float32_matmul_precision('medium')

# Run test and get metrics
test_results = trainer.test(model, test_loader)


/mnt/cephfs/store/gr-mc2473/lc865/misc/condaforge/envs/ml-gnn/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/cephfs/store/gr-mc2473/lc865/misc/condaforge/en ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing DataLoader 0: 100%|██████████| 436/436 [01:05<00:00,  6.62it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test/all-mae-2          19.80620574951172
     test/all-mae-4         22.884592056274414
     test/all-mae-6         25.942747116088867
    test/all-mae-all        22.066444396972656
     test/all-mape-2        52.961570739746094
     test/all-mape-4         58.9210090637207
     test/all-mape-6         68.81634521484375
    test/all-mape-all        58.20038604736328
  test/all-missing_rate     0.12114646285772324
     test/all-rmse-2         29.59722900390625
     test/all-rmse-4        33.648860931396484
     test/all-rmse-6        37.515438079833984
    test/all-rmse-all       32.869049072265625
        test/loss           0.3